In [ ]:
%%capture
%load_ext autoreload
%autoreload 2
%matplotlib inline
%load_ext training_rl
%set_random_seed 12

In [ ]:
%presentation_style

In [ ]:
%load_latex_macros

In [ ]:
%autoreload
from training_rl.offline_rl.load_env_variables import load_env_variables
load_env_variables()


import os
import random
import warnings

import gymnasium as gym
import minari
import numpy as np
from minari import DataCollectorV0, StepDataCallback

from training_rl.offline_rl.custom_envs.custom_2d_grid_env.obstacles_2D_grid_register import \
    ObstacleTypes
from training_rl.offline_rl.custom_envs.custom_2d_grid_env.simple_grid import \
    Custom2DGridEnv
from training_rl.offline_rl.custom_envs.custom_envs_registration import (
    CustomEnv, RenderMode, register_grid_envs)
from training_rl.offline_rl.custom_envs.utils import (
    Grid2DInitialConfig, InitialConfigCustom2DGridEnvWrapper)
from training_rl.offline_rl.generate_custom_minari_datasets.utils import \
    generate_compatible_minari_dataset_name
from training_rl.offline_rl.offline_policies.offpolicy_rendering import \
    offpolicy_rendering
from training_rl.offline_rl.utils import (delete_minari_data_if_exists,
                                          load_buffer_minari,
                                          one_hot_to_integer,
                                          state_action_histogram)
from training_rl.offline_rl.visualizations.utils import (
    get_state_action_data_and_policy_grid_distributions, snapshot_env)
from training_rl.offline_rl.utils import widget_list
from training_rl.offline_rl.behavior_policies.behavior_policy_registry import BehaviorPolicyType
from training_rl.offline_rl.custom_envs.custom_envs_registration import EnvFactory
from training_rl.offline_rl.generate_custom_minari_datasets.generate_minari_dataset_grid_envs import \
    create_minari_datasets


warnings.filterwarnings("ignore")
# To get access to the registered environments.
register_grid_envs()

render_mode = RenderMode.RGB_ARRAY_LIST if os.environ.get("DISPLAY") else None

<img src="_static/images/aai-institute-cover.svg" alt="Snow" style="width:100%;">
<div class="md-slide title"> Minari Overview </div>

# Exercise: Minari data collection

In this exercise you don't have any homework. The idea is to play around with it to get familiar with the code  (notebooks and source code) and with the way we collect data.

Remember that the pipeline for offline learning will be the following:

<img src="_static/images/93_offline_RL_pipeline.png" alt="Snow" style="width:50%;">

In this notebook we will be exploring the steps 1-2-3 . It would be a good idea now to give a look to the code structure:

<img src="_static/images/93_code_structure.png" alt="Snow" style="width:20%;">

You can also give a look to [Minari documentation](https://minari.farama.org/main/content/basic_usage/) if needed.

## STEP 1: Create the environments

In [ ]:
# List of different environments 
ENV_LIST = [
    EnvFactory.Grid_2D_8x8_discrete,
    "InvertedPendulum-v4", 
    "Humanoid-v4", 
    "AdroitHandHammer-v1",
    "HalfCheetah-v4",
]

# obstacles to be used with the 2d grid-world
grid_world_obstacles = [
    ObstacleTypes.door_object_8x8,
    ObstacleTypes.obst_free_8x8, 
]

# behavior policies to be used with 2d grid-world
behavior_policy_grid_world = [
    BehaviorPolicyType.behavior_8x8_suboptimal_initial_0_0_final_0_7,
    BehaviorPolicyType.random,
    BehaviorPolicyType.behavior_8x8_grid_deterministic_0_0_to_4_7,
]

In [ ]:
selected_environment = widget_list(ENV_LIST, description="Mixed envs.")
selected_obstacle = widget_list(grid_world_obstacles, description="grid obst." )
selected_grid_world_policy = widget_list(behavior_policy_grid_world, description="grid policy")

**Select and render behavior policies**

In [ ]:
behavior_policy = BehaviorPolicyType.random
if isinstance(selected_environment.value, EnvFactory):
    env = EnvFactory[selected_environment.value].get_env(render_mode=RenderMode.RGB_ARRAY_LIST)
    if isinstance(env.unwrapped, Custom2DGridEnv):
        env.set_new_obstacle_map(selected_obstacle.value.value)
        behavior_policy = selected_grid_world_policy.value
else:
    env = gym.make(selected_environment.value, render_mode="rgb_array_list")
    
offpolicy_rendering(
    env_or_env_name=env,
    render_mode=RenderMode.RGB_ARRAY_LIST,
    behavior_policy_name=BehaviorPolicyType.random,
    num_frames=100,
    fps=10,
)

## STEP 2: Create Minari datasets

In [ ]:
DATA_SET_IDENTIFIER_I = "_collected_data_nb_92"
NUM_STEPS_I = 500

data_set_config = create_minari_datasets(
    env_name=env.unwrapped.spec.id,
    dataset_identifier=DATA_SET_IDENTIFIER_I,
    num_colected_points=NUM_STEPS_I,
    behavior_policy_name=behavior_policy,
)

### STEP 3: Feed dataset to Tianshou ReplayBuffer

In [ ]:
buffer_data = load_buffer_minari(data_set_config.data_set_name)
print(f"The number of dataset points is {len(buffer_data)}")

for elem in buffer_data:
    print(elem)
    break

**Let's collect and visualize the 2d grid world data with one of its behavior policies**

**Useful information:**

In our grid world environment, the agent's position is represented as $(x_1, x_2)$, with $x_1/x_2$ the vertical/horizontal coordinates. Observations are represented as a one-hot encoded vector of dimensions $x_1 \times x_2$, for example, a 64-dimensional vector in an 8x8 grid,
 

The action is represented by an integer in the range of [0, 1, 2, 3], each indicating a direction:

    0: (-1, 0) - UP
    1: (1, 0) - DOWN
    2: (0, -1) - LEFT
    3: (0, 1) - RIGHT



In [ ]:
# Compute state-action data distribution
if isinstance(env.unwrapped, Custom2DGridEnv):
    state_action_count_data, _ = get_state_action_data_and_policy_grid_distributions(buffer_data, env)
    new_keys = [(env.to_xy(state_action[0]), state_action[1]) for state_action in list(state_action_count_data.keys())]
    state_action_histogram(state_action_count_data, title="State-Action data distribution")
    snapshot_env(env)
else:
    raise ValueError(f"To analyze the data the environment should be of type {Custom2DGridEnv}.")

## References

[ \[Fu.Justin et. al.\] D4RL: Datasets for Deep Data-Driven Reinforcement Learning](https://arxiv.org/abs/2004.07219)

[ MINARI: A dataset API for Offline Reinforcement Learning ](https://minari.farama.org/main/content/basic_usage/)